In [1]:
!pip install gradio >installs.txt
!pip install langchain >installs.txt
!pip install pypdf >installs.txt
!pip install --quiet langchain_experimental python-dotenv langchain_openai langchainhub >installs.txt
!pip install chromadb >installs.txt



In [3]:
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_experimental.text_splitter import SemanticChunker
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains import RetrievalQA
from langchain.agents import AgentExecutor, create_react_agent
from langchain import hub
from langchain.tools.retriever import create_retriever_tool
import gradio as gr
from datetime import datetime
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

In [5]:
def process_pdf(pdf_path):
        loader = PyPDFLoader(pdf_path)
        pages = loader.load_and_split()
        text_splitter = SemanticChunker(OpenAIEmbeddings())
        texts = text_splitter.split_documents(pages)
        vectorstore = Chroma.from_documents(documents=texts, embedding = OpenAIEmbeddings(),persist_directory="./chroma_db")
        vectorstore.persist()
        print("stored inn chroma db")
        retriever = vectorstore.as_retriever()
        return "Your File is Uploaded Successfully!"
chat_history = []


retriever = process_pdf("RAG Input Doc.pdf")



stored inn chroma db


/home/musab/Task1_mini_rag/.venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [6]:


def rag_inference(question):
    vectorstore = Chroma(persist_directory="./chroma_db",embedding_function=OpenAIEmbeddings())
    # print("vectorstore",vectorstore._collection.count())
    retriever = vectorstore.as_retriever(search_kwargs={"k": vectorstore._collection.count()})
    llm = ChatOpenAI(api_key=api_key,model_name="gpt-4-turbo", temperature=0)
    prompt = hub.pull("hwchase17/react")
    tool_search = create_retriever_tool(
        retriever=retriever,
        name="search_document",
        description="Searches and returns excerpts from the document.",
    )
    react_agent = create_react_agent(llm, [tool_search], prompt)
    agent_executor = AgentExecutor(
    agent=react_agent, 
    tools=[tool_search],
    verbose=True,
    handle_parsing_errors=True,
    max_iterations = 10 # useful when agent is stuck in a loop
    )
    # result=qa({"query": question})
    result=agent_executor.invoke({"input": question})
    return result['output'] 


In [7]:
queries = [
    "Which paper received the highest number of stars per hour?",
    "What is the focus of the 'MeshAnything' project?",
    "Which paper discusses the integration of Large Language Models with Monte Carlo Tree Search?",
    "What advancements does the 'VideoLLaMA 2' paper propose?",
    "Which paper was published most recently?",
    "Identify a paper that deals with language modeling and its scalability.",
    "Which paper aims at improving accuracy in Google-Proof Question Answering?",
    "List the categories covered by the paper titled 'TextGrad: Automatic \"Differentiation\" via Text'."
]
all_results=[]
for q in queries:
    result=rag_inference(q)
    print(q)
    print(result)
    print("-----------------------")
    
    all_results.append({'query':q,'result':result})



> Entering new AgentExecutor chain...
To find the paper that received the highest number of stars per hour, I need to access data that shows the number of stars received by various papers and the time period over which they were accumulated.

Action: search_document
Action Input: "papers with the highest number of stars per hour"Stats:  485, 2.04 stars / hour  
Categories:  Question Answering, Specificity  
Links:  Paper, Code  
 
Title:  Scalable MatMul -free Language Modeling  
Authors:  ridgerchu/matmulfreellm  
Date:  4 Jun 2024  
Description:  Our experiments show that our proposed MatMul -free models achieve performance 
on-par with state -of-the-art Transformers that require far more memory during inference at a 
scale up to at least 2.7B parameters. Stats:  2,140, 1.98 stars / hour

Stats:  485, 2.04 stars / hour  
Categories:  Question Answering, Specificity  
Links:  Paper, Code  
 
Title:  Scalable MatMul -free Language Modeling  
Authors:  ridgerchu/matmulfreellm  
Date: 

In [8]:
print(all_results)

[{'query': 'Which paper received the highest number of stars per hour?', 'result': 'The paper titled "MeshAnything: Artist-Created Mesh Generation with Autoregressive Transformers" by buaacyw/meshanything, dated 14 Jun 2024, received the highest number of stars per hour at 5.09 stars/hour.'}, {'query': "What is the focus of the 'MeshAnything' project?", 'result': "The focus of the 'MeshAnything' project is on generating high-quality 3D assets using autoregressive transformers, aiming to match and potentially replace manually crafted assets in various applications."}, {'query': 'Which paper discusses the integration of Large Language Models with Monte Carlo Tree Search?', 'result': 'The paper discussing the integration of Large Language Models with Monte Carlo Tree Search is titled "Accessing GPT-4 level Mathematical Olympiad Solutions via Monte Carlo Tree Self-refine with LLaMa-38B" by trotsky1997/mathblackbox.'}, {'query': "What advancements does the 'VideoLLaMA 2' paper propose?", 'r